<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# NRPy+ SymPy LaTeX Interface (NRPyLaTeX)

## Author: Ken Sible

### The following notebook will demonstrate LaTeX to SymPy conversion, including [Einstein notation](https://en.wikipedia.org/wiki/Einstein_notation).

<a id='top'></a>

# Table of Contents
$$\label{toc}$$

- [Step 1](#step_1): Lexical Analysis and Syntax Analysis
- [Step 2](#step_2): Grammar Demonstration and Sandbox
- [Step 3](#step_3): Tensor Support with Einstein Notation
    - [Example 1](#example_1): Tensor Contraction
    - [Example 2](#example_2): Index Raising
    - [Example 3](#example_3): Cross Product
    - [Example 4](#example_4): Covariant Derivative
    - [Example 5 (1)](#example_5_1): Schwarzschild Metric
    - [Example 5 (2)](#example_5_2): Kretschmann Scalar
    - [Example 6 (1)](#example_6_1): Extrinsic Curvature (ADM Formalism)
    - [Example 6 (2)](#example_6_2): Hamiltonian/Momentum Constraint
- [Step 4](#step_4): Exception Handling and Index Checking
- [Step 5](#step_5): Output Notebook to PDF

Further Reading: [Parsing BSSN (Cartesian) Notebook](Tutorial-LaTeX_Interface_Example-BSSN_Cartesian.ipynb)

<a id='step_1'></a>
## Step 1: Lexical Analysis and Syntax Analysis [ [^](#top) ]

In the following section, we discuss [lexical analysis](https://en.wikipedia.org/wiki/Lexical_analysis) (lexing) and [syntax analysis](https://en.wikipedia.org/wiki/Parsing) (parsing). In lexical analysis, a lexical analyzer (or lexer) can tokenize a character string, called a sentence, using substring pattern matching. In syntax analysis, a syntax analyzer (or parser) can construct a parse tree, containing all syntactic information of the language (specified by a [formal grammar](https://en.wikipedia.org/wiki/Formal_grammar)), after receiving a token iterator from the lexical analyzer.

For LaTeX to SymPy conversion, we implemented a [recursive descent parser](https://en.wikipedia.org/wiki/Recursive_descent_parser) that can construct a parse tree in [preorder traversal](https://en.wikipedia.org/wiki/Tree_traversal#Pre-order_(NLR)), starting from the root [nonterminal](https://en.wikipedia.org/wiki/Terminal_and_nonterminal_symbols), using a [right recursive](https://en.wikipedia.org/wiki/Left_recursion) grammar (partially shown below in the canonical (extended) [BNF](https://en.wikipedia.org/wiki/Extended_Backus%E2%80%93Naur_form) notation).

```
<EXPRESSION>    -> <TERM> { ( '+' | '-' ) <TERM> }*
<TERM>          -> <FACTOR> { [ '/' ] <FACTOR> }*
<FACTOR>        -> <BASE> { '^' <EXPONENT> }*
<BASE>          -> [ '-' ] ( <ATOM> | <SUBEXPR> )
<EXPONENT>      -> <BASE> | '{' <BASE> '}' | '{' '{' <BASE> '}' '}'
<ATOM>          -> <COMMAND> | <OPERATOR> | <NUMBER> | <TENSOR>
<SUBEXPR>       -> '(' <EXPRESSION> ')' | '[' <EXPRESSION> ']' | '\' '{' <EXPRESSION> '\' '}'
<COMMAND>       -> <FUNC> | <FRAC> | <SQRT> | <NLOG> | <TRIG>
    ⋮            ⋮
```

<small>**Source**: Robert W. Sebesta. Concepts of Programming Languages. Pearson Education Limited, 2016.</small>

In [1]:
import sympy as sp
try: import nrpylatex
except ImportError:
    !pip install nrpylatex
!pip freeze | grep nrpylatex
from nrpylatex import *

nrpylatex==1.0.7


In [2]:
lexer = Lexer(); lexer.initialize(r'(1 + x/n)^n')
print(', '.join(token for token in lexer.tokenize()))

LPAREN, INTEGER, PLUS, LETTER, DIVIDE, LETTER, RPAREN, CARET, LETTER


In [3]:
expr = parse_latex(r'(1 + x/n)^n')
print(expr, '\n  >>', sp.srepr(expr))

(1 + x/n)**n 
  >> Pow(Add(Integer(1), Mul(Pow(Symbol('n', real=True), Integer(-1)), Symbol('x', real=True))), Symbol('n', real=True))


`Grammar Derivation: (1 + x/n)^n`
```
<EXPRESSION> -> <TERM>
             -> <FACTOR>
             -> <BASE>^<EXPONENT>
             -> <SUBEXPR>^<EXPONENT>
             -> (<EXPRESSION>)^<EXPONENT>
             -> (<TERM> + <TERM>)^<EXPONENT>
             -> (<FACTOR> + <TERM>)^<EXPONENT>
             -> (<BASE> + <TERM>)^<EXPONENT>
             -> (<ATOM> + <TERM>)^<EXPONENT>
             -> (<NUMBER> + <TERM>)^<EXPONENT>
             -> (<INTEGER> + <TERM>)^<EXPONENT>
             -> (1 + <TERM>)^<EXPONENT>
             -> (1 + <FACTOR> / <FACTOR>)^<EXPONENT>
             -> ...
```

<a id='step_2'></a>
## Step 2: Grammar Demonstration and Sandbox [ [^](#top) ]

In the following section, we demonstrate the process for extending the parsing module to include a (previously) unsupported LaTeX command.

1. update the `grammar` dictionary in the `Lexer` class with the mapping `regex` $\mapsto$ `token`
1. write a grammar abstraction in BNF notation (similar to a regular expression) for the command
1. implement a private method for the nonterminal (command name) to parse the grammar abstraction

```<SQRT> -> <SQRT_CMD> [ '[' <INTEGER> ']' ] '{' <EXPRESSION> '}'```
```
def _sqrt(self):
    self.expect('SQRT_CMD')
    if self.accept('LBRACK'):
        integer = self.lexer.lexeme
        self.expect('INTEGER')
        root = Rational(1, integer)
        self.expect('RBRACK')
    else: root = Rational(1, 2)
    self.expect('LBRACE')
    expr = self._expression()
    self.expect('RBRACE')
    if root == Rational(1, 2):
        return sqrt(expr)
    return Pow(expr, root)
```

In addition to expression parsing, we included support for equation parsing, which can produce a dictionary mapping `LHS` $\mapsto$ `RHS`, where `LHS` must be a symbol, and insert that mapping into the global namespace of the previous stack frame, as demonstrated below.

$$ \mathit{s_n} = \left(1 + \frac{1}{n}\right)^n $$

In [4]:
parse_latex(r'\text{s_n} = \left(1 + \frac{1}{n}\right)^n')

('s_n',)

In [5]:
print('s_n =', s_n)

s_n = (1 + 1/n)**n


Furthermore, we implemented robust error messaging using the custom `ParseError` exception, which should handle every conceivable case to identify, as detailed as possible, invalid syntax inside of a LaTeX sentence. The following are some runnable examples of possible error messages.

In [6]:
try: parse_latex(r'5x^{{4$}}')
except ParseError as e:
    print(type(e).__name__ + ': ' + str(e))

ParseError: 5x^{{4$}}
                  ^
unexpected '$' at position 6


In [7]:
try: parse_latex(r'\sqrt[0.1]{5x^{{4}}}')
except ParseError as e:
    print(type(e).__name__ + ': ' + str(e))

ParseError: \sqrt[0.1]{5x^{{4}}}
                  ^
expected token INTEGER at position 6


In [8]:
try: parse_latex(r'\int_0^5 5x^{{4}}dx')
except ParseError as e:
    print(type(e).__name__ + ': ' + str(e))

ParseError: \int_0^5 5x^{{4}}dx
            ^
unsupported command '\int' at position 0


In the sandbox code cell below, you can experiment with converting LaTeX to SymPy using the wrapper function `parse(sentence)`, where `sentence` must be a Python [raw string](https://docs.python.org/3/reference/lexical_analysis.html) to interpret a backslash as a literal character rather than an [escape sequence](https://en.wikipedia.org/wiki/Escape_sequence). You could, alternatively, use the supported cell magic `%%parse_latex` to automatically escape every backslash and parse the cell (more convenient than `parse(sentence)` in a notebook format).

In [9]:
# Write Sandbox Code Here

<a id='step_3'></a>
## Step 3: Tensor Support with Einstein Notation [ [^](#top) ]

In the following section, we demonstrate parsing tensor notation using the Einstein summation convention. In each example, every tensor should appear either on the LHS of an equation or on the RHS of a `vardef` macro before appearing on the RHS of an equation. Furthermore, an exception will be raised upon violation of the Einstein summation convention, i.e. the occurrence of an invalid free or bound index.

**Configuration Grammar**

```
<MACRO>     -> <PARSE> | <SREPL> | <VARDEF> | <KEYDEF> | <ASSIGN> | <IGNORE>
<PARSE>     -> <PARSE_MACRO> <ASSIGNMENT> { ',' <ASSIGNMENT> }*
<SREPL>     -> <SREPL_MACRO> <STRING> <ARROW> <STRING> { ',' <STRING> <ARROW> <STRING> }*
<VARDEF>    -> <VARDEF_MACRO> { '-' <OPTION> }* <VARIABLE> { ',' <VARIABLE> }* [ '(' <DIMENSION> ')' ]
<KEYDEF>    -> <KEYDEF_MACRO> <BASIS_KWRD> <BASIS> | <INDEX_KWRD> <INDEX>
<ASSIGN>    -> <ASSIGN_MACRO> { '-' <OPTION> }* <VARIABLE> { ',' <VARIABLE> }*
<IGNORE>    -> <IGNORE_MACRO> <STRING> { ',' <STRING> }*
<OPTION>    -> <DRV_TYPE> [ <PRIORITY> ] | <SYMMETRY> | <WEIGHT> '=' <NUMBER>
<BASIS>     -> <BASIS_KWRD> '{' <LETTER> { ',' <LETTER> }* '}'
<INDEX>     -> ( <LETTER> | '[' <LETTER> '-' <LETTER> ']' ) '(' <DIMENSION> ')'
```

<a id='example_1'></a>
### Example 1. [Tensor Contraction](https://en.wikipedia.org/wiki/Tensor_contraction) [ [^](#top) ]

In [10]:
parse_latex(r"""
    % vardef 'hUD' (4D)
    h = h^\mu{}_\mu
""", reset=True, verbose=True)

(Tensor(hUD, 4D), Scalar(h))

In [11]:
print('h =', h)

h = hUD00 + hUD11 + hUD22 + hUD33


<a id='example_2'></a>
### Example 2. [Index Raising](https://en.wikipedia.org/wiki/Raising_and_lowering_indices) [ [^](#top) ]

In [12]:
parse_latex(r"""
    % vardef -metric 'gUU' (3D)
    % vardef 'vD' (3D)
    v^\mu = g^{\mu\nu} v_\nu
""", reset=True)

('gUU', 'epsilonDDD', 'gdet', 'gDD', 'GammaUDD', 'vD', 'vU')

In [13]:
print('vU =', vU)

vU = [gUU00*vD0 + gUU01*vD1 + gUU02*vD2, gUU01*vD0 + gUU11*vD1 + gUU12*vD2, gUU02*vD0 + gUU12*vD1 + gUU22*vD2]


<a id='example_3'></a>
### Example 3. [Cross Product](https://en.wikipedia.org/wiki/Cross_product) [ [^](#top) ]

In [14]:
parse_latex(r"""
    % vardef 'vU' (3D), 'wU' (3D)
    u_i = \epsilon_{ijk} v^j w^k
""", reset=True)

('vU', 'wU', 'epsilonDDD', 'uD')

In [15]:
print('uD =', uD)

uD = [vU1*wU2 - vU2*wU1, -vU0*wU2 + vU2*wU0, vU0*wU1 - vU1*wU0]


<a id='example_4'></a>
### Example 4. [Covariant Derivative](https://en.wikipedia.org/wiki/Covariant_derivative) [ [^](#top) ]

The following are contextually inferred, dynamically generated, and injected into the global namespace for expansion of the covariant derivative $\nabla_\nu F^{\mu\nu}$
$$
\begin{align*}
    \Gamma^\mu_{ba} &= \frac{1}{2} g^{\mu c}(\partial_b\,g_{a c} + \partial_a\,g_{c b} - \partial_c\,g_{b a}) \\
    \Gamma^\nu_{ba} &= \frac{1}{2} g^{\nu c}(\partial_b\,g_{a c} + \partial_a\,g_{c b} - \partial_c\,g_{b a}) \\
    \nabla_a F^{\mu \nu} &= \partial_a F^{\mu \nu} + \Gamma^\mu_{b a} F^{b \nu} + \Gamma^\nu_{b a} F^{\mu b}
\end{align*}
$$

In [16]:
parse_latex(r"""
    % vardef -diff_type=dD -symmetry=anti01 'FUU' (4D)
    % vardef -diff_type=dD -metric 'gDD' (4D)
    % vardef -const 'k'
    J^\mu = (4\pi k)^{-1} \nabla_\nu F^{\mu\nu}
""", reset=True)

('FUU',
 'gDD',
 'epsilonUUUU',
 'gdet',
 'gUU',
 'gDD_dD',
 'GammaUDD',
 'FUU_dD',
 'FUU_cdD',
 'JU')

In [17]:
parse_latex(r"""
    % vardef -diff_type=dD -symmetry=anti01 'FUU' (4D)
    % vardef -diff_type=dD -metric 'ghatDD' (4D)
    % vardef -const 'k'
    J^\mu = (4\pi k)^{-1} \vphantom{dD} \hat{\nabla}_\nu F^{\mu\nu}
""", reset=True)

('FUU',
 'ghatDD',
 'epsilonUUUU',
 'ghatdet',
 'ghatUU',
 'ghatDD_dD',
 'GammahatUDD',
 'FUU_dD',
 'FUU_cdhatD',
 'JU')

<a id='example_5_1'></a>
### Example 5 (1). [Schwarzschild Metric](https://en.wikipedia.org/wiki/Schwarzschild_metric) [ [^](#top) ]

In [18]:
%load_ext nrpylatex.extension

In [19]:
%%parse_latex --reset --ignore-warning

% keydef basis [t, r, \theta, \phi]
% vardef -zero 'gDD' (4D)
% vardef -const 'G', 'M'
\begin{align}
    g_{t t} &= -\left(1 - \frac{2GM}{r}\right) \\
    g_{r r} &=  \left(1 - \frac{2GM}{r}\right)^{-1} \\
    g_{\theta \theta} &= r^2 \\
    g_{\phi \phi} &= r^2 \sin^2\theta
\end{align}
% assign -metric 'gDD'

('gDD', 'epsilonUUUU', 'gdet', 'gUU', 'GammaUDD')

In [20]:
sp.Matrix(gDD)

Matrix([
[2*G*M/r - 1,                0,    0,                  0],
[          0, 1/(-2*G*M/r + 1),    0,                  0],
[          0,                0, r**2,                  0],
[          0,                0,    0, r**2*sin(theta)**2]])

<a id='example_5_2'></a>
### Example 5 (2). [Kretschmann Scalar](https://en.wikipedia.org/wiki/Kretschmann_scalar) [ [^](#top) ]

In [21]:
%%parse_latex

\begin{align}
    R^\alpha{}_{\beta\mu\nu} &= \partial_\mu \Gamma^\alpha_{\beta\nu} - \partial_\nu \Gamma^\alpha_{\beta\mu}
        + \Gamma^\alpha_{\mu\gamma}\Gamma^\gamma_{\beta\nu} - \Gamma^\alpha_{\nu\sigma}\Gamma^\sigma_{\beta\mu} \\
    K &= R^{\alpha\beta\mu\nu} R_{\alpha\beta\mu\nu} \\
    R_{\beta\nu} &= R^\alpha{}_{\beta\alpha\nu} \\
    R &= g^{\beta\nu} R_{\beta\nu} \\
    G_{\beta\nu} &= R_{\beta\nu} - \frac{1}{2}g_{\beta\nu}R
\end{align}

('RUDDD', 'RUUUU', 'RDDDD', 'K', 'RDD', 'R', 'GDD')

In [22]:
sp.simplify(sp.Matrix(RDD))

Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0]])

In [23]:
display(sp.Matrix(GammaUDD[0][:][:]))

Matrix([
[                        0, -G*M/(r**2*(2*G*M/r - 1)), 0, 0],
[-G*M/(r**2*(2*G*M/r - 1)),                         0, 0, 0],
[                        0,                         0, 0, 0],
[                        0,                         0, 0, 0]])

In [24]:
display(sp.Matrix(GammaUDD[1][:][:]))

Matrix([
[G*M*(-2*G*M/r + 1)/r**2,                          0,                 0,                               0],
[                      0, -G*M/(r**2*(-2*G*M/r + 1)),                 0,                               0],
[                      0,                          0, -r*(-2*G*M/r + 1),                               0],
[                      0,                          0,                 0, -r*(-2*G*M/r + 1)*sin(theta)**2]])

In [25]:
display(sp.Matrix(GammaUDD[2][:][:]))

Matrix([
[0,   0,   0,                      0],
[0,   0, 1/r,                      0],
[0, 1/r,   0,                      0],
[0,   0,   0, -sin(theta)*cos(theta)]])

In [26]:
display(sp.Matrix(GammaUDD[3][:][:]))

Matrix([
[0,   0,                     0,                     0],
[0,   0,                     0,                   1/r],
[0,   0,                     0, cos(theta)/sin(theta)],
[0, 1/r, cos(theta)/sin(theta),                     0]])

For the Schwarzschild metric, the Kretschmann scalar $K$ has the property that $K\to\infty$ as $r\to 0$, and hence the metric and spacetime itself are undefined at the point of infinite curvature $r=0$, indicating the presence of a physical singularity since the Kretschmann scalar is an [invariant quantity](https://en.wikipedia.org/wiki/Curvature_invariant_(general_relativity)) in general relativity.

In [27]:
display(sp.simplify(K))

48*G**2*M**2/r**6

In a [vacuum region](https://en.wikipedia.org/wiki/Vacuum_solution_(general_relativity)#:~:text=In%20general%20relativity%2C%20a%20vacuum,non%2Dgravitational%20fields%20are%20present.), such as the spacetime described by the Schwarzschild metric, $T_{\mu\nu}=0$ and hence $G_{\mu\nu}=0$ since $G_{\mu\nu}=8\pi G\,T_{\mu\nu}$ ([Einstein Equations](https://en.wikipedia.org/wiki/Einstein_field_equations)).

In [28]:
sp.simplify(sp.Matrix(GDD))

Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0]])

<a id='example_6_1'></a>
### Example 6 (1). [Extrinsic Curvature](https://en.wikipedia.org/wiki/Curvature) ([ADM Formalism](https://en.wikipedia.org/wiki/ADM_formalism)) [ [^](#top) ]

In [29]:
%%parse_latex --ignore-warning

% keydef basis [r, \theta, \phi]
\begin{align}
    \gamma_{ij} &= g_{ij} \\
    % assign -metric 'gammaDD'
    \beta_i &= g_{r i} \\
    \alpha &= \sqrt{\gamma^{ij}\beta_i\beta_j - g_{r r}} \\
    K_{ij} &= \frac{1}{2\alpha}\left(\nabla_i \beta_j + \nabla_j \beta_i\right) \\
    K &= \gamma^{ij} K_{ij}
\end{align}

('gammaDD',
 'epsilonUUU',
 'gammadet',
 'gammaUU',
 'betaD',
 'alpha',
 'betaD_cdD',
 'KDD')

For the Schwarzschild metric (defined in the previous example), the extrinsic curvature in the ADM formalism should evaluate to zero.

In [30]:
display(sp.Matrix(KDD))

Matrix([
[0, 0, 0],
[0, 0, 0],
[0, 0, 0]])

<a id='example_6_2'></a>
### Example 6 (2). [Hamiltonian/Momentum Constraint](https://en.wikipedia.org/wiki/Hamiltonian_constraint) [ [^](#top) ]

In [31]:
%%parse_latex --ignore-warning

\begin{align}
    R_{ij} &= \partial_k \Gamma^k_{ij} - \partial_j \Gamma^k_{ik}
        + \Gamma^k_{ij}\Gamma^l_{kl} - \Gamma^l_{ik}\Gamma^k_{lj} \\
    R &= \gamma^{ij} R_{ij} \\
    E &= \frac{1}{16\pi}\left(R + K^{{2}} - K_{ij}K^{ij}\right) \\
    p_i &= \frac{1}{8\pi}\left(D_j \gamma^{jk} K_{ki} - D_i K\right)
\end{align}

('KUU', 'E', 'gammaUU_cdD', 'K_cdD', 'pD')

Every solution to the Einstein Equations, including Schwarzschild, must satisfy the Hamiltonian constraint ($E=0$) and the Momentum constraint ($p_i=0$).

In [32]:
print('E = %s, pD = %s' % (sp.simplify(E), pD))

E = 0, pD = [0, 0, 0]


<a id='step_4'></a>
## Step 4: Exception Handling and Index Checking ( [^](#top) )

We extended our robust error messaging using the custom `TensorError` exception, which should handle any inconsistent tensor dimension and any violation of the Einstein summation convention, specifically that a bound index must appear exactly once as a superscript and exactly once as a subscript in any single term and that a free index must appear in every term with the same position and cannot be summed over in any term.

In [33]:
%%parse_latex --reset

% vardef 'TUD' (4D), 'uD' (4D)
v^\mu = T^\mu_\nu u_\nu

TensorError: illegal bound index 'nu' in vU


In [34]:
%%parse_latex --reset

% vardef 'TUD' (4D), 'uD' (4D)
v^\mu = T^\mu_\nu u_\mu

TensorError: unbalanced free index {'mu', 'nu'} in vU


In [35]:
%%parse_latex --reset

% vardef 'TUD' (4D), 'uD' (3D)
v^\mu = T^\mu_\nu u_\mu

ParseError: inconsistent dimension for index 'mu'


In [36]:
%%parse_latex --reset

% vardef 'vD' (4D)
T_{\mu\nu} = v_\mu w_\nu

ParseError: T_{\mu\nu} = v_\mu w_\nu
                               ^
cannot index undefined tensor 'wD' at position 38


In [37]:
%%parse_latex --reset

% vardef -symmetry=anti01 'FUU' (4D)
% vardef -const 'k'
J^\mu = (4\pi k)^{-1} \nabla_\nu F^{\mu\nu}

ParseError: J^\mu = (4\pi k)^{-1} \nabla_\nu F^{\mu\nu}
                                  ^
cannot generate covariant derivative without defined metric 'g'


<a id='step_5'></a>
## Step 5: Output Notebook to PDF ( [^](#top) )

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-SymPy_LaTeX_Interface.pdf](Tutorial-SymPy_LaTeX_Interface.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [38]:
import cmdline_helper as cmd    # NRPy+: Multi-platform Python command-line interface
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("Tutorial-SymPy_LaTeX_Interface")

Created Tutorial-SymPy_LaTeX_Interface.tex, and compiled LaTeX file to PDF
    file Tutorial-SymPy_LaTeX_Interface.pdf
